### Install necessary libraries

In [1]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 3.0 MB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 7.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.4 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 4.6 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 4.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 2. Prepare the dataset

In [3]:
import pandas as pd

df = pd.read_csv('../archive/cleaned/for_training_no_nulls.csv')

# Clean and prepare
df.dropna(subset=['title', 'description', 'genre', 'rating'], inplace=True)

df['rating'] = df['rating'].astype(str)
df['genre'] = df['genre'].str.lower().str.strip()

# Create task inputs
df_rating = df.copy()
df_rating['input_text'] = "predict rating: Title: " + df['title'] + " Description: " + df['description']
df_rating['target_text'] = df['rating']

df_genre = df.copy()
df_genre['input_text'] = "predict genre: Title: " + df['title'] + " Description: " + df['description']
df_genre['target_text'] = df['genre']

# Combine both tasks
final_df = pd.concat([df_rating[['input_text', 'target_text']], df_genre[['input_text', 'target_text']]])
final_df = final_df.sample(frac=1).reset_index(drop=True)



### Load and fine tune T5

In [1]:
!pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenize dataset
def preprocess(example):
    input_enc = tokenizer(example['input_text'], padding="max_length", truncation=True, max_length=512)
    target_enc = tokenizer(example['target_text'], padding="max_length", truncation=True, max_length=64)
    return {
        'input_ids': input_enc['input_ids'],
        'attention_mask': input_enc['attention_mask'],
        'labels': target_enc['input_ids']
    }

dataset = Dataset.from_pandas(final_df)
tokenized_dataset = dataset.map(preprocess)

# Define training args
training_args = TrainingArguments(
    output_dir="./t5-multitask",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    # evaluation_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Train
trainer.train()

Map:   0%|          | 0/102280 [00:00<?, ? examples/s]

/tmp/ipykernel_21/3034974924.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


In [ ]:
# Define training args
training_args = TrainingArguments(
    output_dir="./t5-multitask",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    # evaluation_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Train
trainer.train()